<a href="https://colab.research.google.com/github/NbtKmy/gc_workshops/blob/main/LangChain_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Was ist LangChain?

LangChain ist ein Framework, durch das die Entwicklung einer Applikation mit LLMs (Large Language Models) erleichtert wird. 
LangChain bieten beispielsweise folgende Module an: 

| Modul | Funktion |
|-------|----------|
| Indexes | Indexes wird verwendet, um Texte als Informationsquelle zu einem LLM zu liefern. |
| Memory | Memory speichert die Kommunikationshistorie mit dem LLM. |
| Agents | Agents bieten Zugriffe auf weitere Informationsquelle durch API. |


Durch diese Funktionen kann man eigene Textdaten (zum Beispiel PDF-Dokument oder Text-Datei) oder Wikipedia-Einträge als Informationsquelle für eigenes ChatBot verwenden

## Embedding mit einem TEI-Daten


In [ ]:
!pip install -q langchain openai faiss-cpu requests bs4 lxml tiktoken

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "[your OpenAI-API-Key]"

### Zum Quellentext

In diesem Beispiel verwenden wir Kafkas "Josefine" aus dem Textgrid: 
 
 >TextGrid Repository (2012). Kafka, Franz. Josefine, die Sängerin. Digitale Bibliothek. https://hdl.handle.net/11858/00-1734-0000-0003-8DA9-7

 Diese TEI-Datei ist unter der Lizenz [CC 3.0](https://creativecommons.org/licenses/by/3.0/de/) freigegeben. 

 Die TEI-Datei ist auch [hier](https://textgridlab.org/1.0/tgcrud-public/rest/textgrid:qmv3.0/data) direkt zu nehmen. Aber im folgenden Code laden wir zuerst die TEI-Datei auf unseren eigenen Laptop herunter, dann diese Datei wieder hochladen. 

In [ ]:
from bs4 import BeautifulSoup
from google.colab import files
from langchain.schema import Document
import re

uploaded = files.upload()
filename = next(iter(uploaded))

data = ""
with open(filename) as f:
    data = f.read()

soupObj = BeautifulSoup(data, "lxml-xml")

titleStmt = soupObj.find("titleStmt")
title = titleStmt.find("title").text
if soupObj.find("notesStmt"):
    notesStmt = soupObj.find("notesStmt")
    note = notesStmt.find("note").text
else: 
    note = "None"

first_pb = soupObj.find("pb")
first_n = int(first_pb["n"])
paragraphs = soupObj.find("text").find_all("p")
pagenum = first_n

# Dokument-Objekt von LangChain erstellen und in einem Listen-Objekt hineinschieben
def create_document(title, note, pagenum, content, docs):
    text_content = content.replace("\n", "")
    note = note.replace("\n", "")
    note = re.sub(r"\s\s+", "", note)
    if len(text_content) > 0:
        metadata = {"source": title, "note": note, "page": int(pagenum)}
        doc_obj = Document(page_content=content, metadata=metadata)
        docs.append(doc_obj)


docs = []
for paragraph in paragraphs:
    
    paragraph_contents_list = list(paragraph.children)

    if len(paragraph_contents_list) == 3:
        pb_index = paragraph.find("pb")
        pagenum = int(pb_index["n"])
        pagenum_before = pagenum - 1
        
        before_pb = paragraph_contents_list[:paragraph_contents_list.index(pb_index)][0]
        create_document(title, note, pagenum_before, before_pb, docs)

        after_pb = paragraph_contents_list[paragraph_contents_list.index(pb_index) + 1:][0]
        create_document(title, note, pagenum, after_pb, docs)
        
    
    elif len(paragraph_contents_list) > 3:
      
        pb_indexes = paragraph.find_all("pb")
        
        for idx, pb_index in enumerate(pb_indexes): 
            pagenum = int(pb_index["n"])
            sliced_list = paragraph_contents_list[:paragraph_contents_list.index(pb_index)]
            sliced_list_after = paragraph_contents_list[paragraph_contents_list.index(pb_index):]
            next_pb = pb_indexes[(idx + 1) % len(pb_indexes)]

            if len(sliced_list) == 0 or len(sliced_list_after) == 1:
                continue

            elif len(sliced_list) == 1:
                content = sliced_list[0]
                create_document(title, note, pagenum - 1, content, docs)
                content = paragraph_contents_list[paragraph_contents_list.index(pb_index) +1::paragraph_contents_list.index(next_pb)][0]
                create_document(title, note, pagenum, content, docs)

            elif len(sliced_list_after) == 2:
                content = paragraph_contents_list[paragraph_contents_list.index(pb_index) +1:][0]
                create_document(title, note, pagenum, content, docs)
            
            else:
                content = paragraph_contents_list[paragraph_contents_list.index(pb_index) +1::paragraph_contents_list.index(next_pb)][0]
                create_document(title, note, pagenum, content, docs)
    else: 
        paragraph_text = paragraph_contents_list[0]
        create_document(title, note, pagenum, paragraph_text, docs)
        



### Vector-Daten in Vector Stores laden

Mögliche Vector Stores sind [hier](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html#) zu finden.

Hier verwenden wir FAISS.
FAISS ist die Abkürzung von "Facebook AI Similarity Search". Andere Instanzen wie ElasticSearch oder OpenSearch fordern weitere Installation dieser Instanzen. Um LangChain auszuprobieren, ist FAISS hier ausreichend. Wenn man später eine eigene Anwendung aufbauen will, kommen weitere Instanzen in Frage.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain


embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)
query = "Wie ist der Gesang von Josefine?"
answers = db.similarity_search(query)
answers 

### Frage an ChatBot durch LangChain losschicken

#### Parameter

**tempereture** : Temperature kontrolliert die Kreativität des ChatBots. Es gibt Range zwischen 0 und 2. Je höher der Wert ist, desto kreativer wird die Antwort. 
Nähere Information ist im [offiziellen Dokument](https://platform.openai.com/docs/api-reference/chat/create#chat/create-temperature) zu finden.

**chain_type** : Es sind die 4 Chain-Typen möglich; stuff, map_reduce, refine und map_rerank

| Chain-Type | Legende |
|-------------|-----------|
| stuff | Stuffing is the simplest method, whereby you simply stuff all the related data into the prompt as context to pass to the language model.|
| map_reduce | This method involves running an initial prompt on each chunk of data. |
| refine | This method involves running an initial prompt on the first chunk of data, generating some output. |
| map_rerank | This method involves running an initial prompt on each chunk of data, that not only tries to complete a task but also gives a score for how certain it is in its answer. |

[Nähere Beschreibung](https://docs.langchain.com/docs/components/chains/index_related_chains)




In [ ]:
chain = load_qa_chain(OpenAI(temperature=0.1, max_tokens=1000), chain_type="stuff")
query = "Was ist Josefines Singen eigentlich?"
similar_texts = db.similarity_search(query)
# Query mit langChain
chain.run(input_documents=similar_texts, question=query)

' Es ist nicht klar, ob Josefines Singen eigentlich Gesang oder Pfeifen ist.'

## Wikipedia-Einträge einbinden 

LangChain bietet [eine Library "wikipedia"](https://python.langchain.com/en/latest/modules/agents/tools/examples/wikipedia.html) an. Damit kann man den Inhalt der Wikipedia in Bezug nehmen. 

Weil wir die Information in Wikipedia für das ChatBot benutzen möchten, wird das Modul "Agent" verwendet. 


In [ ]:
!pip install -q langchain openai tiktoken wikipedia

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "Dein API Token hier"

In [6]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
import wikipedia

wikipedia.set_lang("de")

llm = OpenAI(temperature=0.2)
tools = load_tools(["wikipedia"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Hier wird die Frage formuliert und abgeschickt.
agent.run("Wann fand Berlinale 2023 statt?")



> Entering new AgentExecutor chain...
 I need to find out when Berlinale 2023 is happening
Action: Wikipedia
Action Input: Berlinale 2023
Observation: Page: 73rd Berlin International Film Festival
Summary: The 73rd annual Berlin International Film Festival, usually called the Berlinale (German pronunciation: [bɛʁliˈnaːlə] (listen)), took place from 16 to 26 February 2023. It was the first completely in-person Berlinale since the 70th in 2020. The festival has added a new award for best television series this year. On 15 December 2022, the first Panorama and Generation titles for the festival were announced, and on 13 January 2023, many world premieres were added to out-of-competition lineup, including Israeli filmmaker Guy Nattiv's Golda—a biographical film about Golda Meir, first female Prime Minister of Israel.The festival opened with American filmmaker and novelist Rebecca Miller's drama film She Came to Me. A live video stream with Ukrainian President Volodymyr Zelenskyy was part

'Die 73. Berlinale fand vom 16. bis 26. Februar 2023 statt.'